In [ ]:
from google.colab import files
uploaded = files.upload()




Saving uszips.xlsx to uszips.xlsx


# New section

# New section

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree

# Load your ZIP code data
df = pd.read_excel("uszips.xlsx")

# Extract and clean lat-lng points
all_points = df[['lat', 'lng']].dropna().drop_duplicates().reset_index(drop=True)

# Define geographic bounding box for continental US
min_lat, max_lat = 24.5, 49.5
min_lng, max_lng = -125, -66.9

# Create a 30x30 grid
grid_size = int(np.sqrt(900))
lat_grid = np.linspace(min_lat, max_lat, grid_size)
lng_grid = np.linspace(min_lng, max_lng, grid_size)
grid_points = np.array([(lat, lng) for lat in lat_grid for lng in lng_grid])

# Find nearest ZIP code point to each grid point
tree = KDTree(all_points[['lat', 'lng']].values)
_, indices = tree.query(grid_points, k=1)

# Collect nearest actual ZIP code points
selected_points = all_points.iloc[indices.flatten()].drop_duplicates().reset_index(drop=True)

# Limit to exactly 900 points if needed
if len(selected_points) > 900:
    selected_points = selected_points.sample(n=900, random_state=42).reset_index(drop=True)

# Save to CSV
selected_points.to_csv("grid_based_900_us_locations.csv", index=False)

print("Saved 900 lat-long points to 'grid_based_900_us_locations.csv'")


Saved 900 lat-long points to 'grid_based_900_us_locations.csv'


In [ ]:
import folium

# Create a base map centered on the USA
map_center = [39.5, -98.35]  # Approximate center of continental USA
m = folium.Map(location=map_center, zoom_start=4)

# Add the 900 points to the map
for _, row in selected_points.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lng']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# Display map
m
